<a href="https://colab.research.google.com/github/thekhush11/GenAI-Colab-Projects/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [ ]:
# Hyperparameter ==

Buf_Size = 60000
Bat_Size = 255
Img_Hi = 28
Img_Wi = 28
Channels = 1
Epochs = 30
Lat_Dim = 100


In [ ]:
# load and Preprocess Dataset
def load_dataset():
    (x_train,y_train),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = x_train.reshape((-1,Img_Hi,Img_Wi,Channels)).astype("float32")
    x_train = (x_train-127.5)/127.5
    return x_train

In [ ]:
x_train = load_dataset()

train_dataset = tf.data.Dataset\
.from_tensor_slices(x_train)\
.shuffle(Buf_Size)\
.batch(Bat_Size)\
.prefetch(tf.data.AUTOTUNE)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Build Generators --
def Build_generatos():
    model = tf.keras.Sequential(name="Generator")
    model.add(layers.Dense(7*7*255,use_bias=False,input_shape=(Lat_Dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Reshape((7,7,255)))

    # UpSample 14X14
    model.add(layers.Conv2DTranspose(128,kernel_size=5,strides=2,padding="same",use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # 28X28
    model.add(layers.Conv2DTranspose(128,kernel_size=5,strides=2,padding="same",use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # Final layers ==
    model.add(layers.Conv2DTranspose(Channels,kernel_size=5,strides=1,padding="same",use_bias=False,activation="tanh"))
    return model

In [ ]:
generator = Build_generatos()
generator.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12495)          │     1,249,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12495)          │        49,980 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 12495)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 255)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 14, 14, 128)    │       816,000 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 28, 28, 128)    │       409,600 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 28, 28, 1)      │         3,200 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,529,304 (9.65 MB)

 Trainable params: 2,503,802 (9.55 MB)

 Non-trainable params: 25,502 (99.62 KB)

In [ ]:
# Build Discriminators ==
def Build_discriminators():
    model = tf.keras.Sequential(name="Discriminator")
    model.add(layers.Conv2D(64,kernel_size=5,strides=2,
                            padding="same",use_bias=False,
                            input_shape=[Img_Hi,Img_Wi,Channels]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128,kernel_size=5,strides=2,
                            padding="same",use_bias=False))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

In [ ]:
discriminator = Build_discriminators()
discriminator.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │       204,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,673 (830.75 KB)

 Trainable params: 212,673 (830.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Losses and Optimizers ==
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def loss_discriminators(real,fake):
    real_loss = cross_entropy(tf.ones_like(real),real)
    fake_loss = cross_entropy(tf.zeros_like(fake),fake)

    return real_loss + fake_loss

def loss_generators(fake):
    return cross_entropy(tf.ones_like(fake),fake)

In [ ]:
# Optimizers ==
generator_optimizers = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizers = tf.keras.optimizers.Adam(1e-4)

In [ ]:
#
Number_of_sample = 15
output_dir = "Generated_image"
checkpoint_dir = "checkpoint"

os.makedirs(output_dir,exist_ok=True)
os.makedirs(checkpoint_dir,exist_ok=True)

In [ ]:
# Input for Consistent inputs --
inputs = tf.random.normal([Number_of_sample,Lat_Dim])

In [ ]:
# Checkpoint --
checkpoint1 = os.path.join(checkpoint_dir,"ckpt")

In [ ]:
checkpoint2 = tf.train.Checkpoint(generator_optimizers=generator_optimizers,
                                  discriminator_optimizers=discriminator_optimizers,
                                  generator=generator,
                                  discriminator=discriminator)

In [ ]:
# Checkpoint manager ==
checkpoint_manager = tf.train.CheckpointManager(checkpoint2,checkpoint_dir,max_to_keep=5)

In [ ]:
# Restore the latest checkpoints



In [ ]:
# Generate & save Image ==
def Gen_and_save_iamge(model,epochs,test_input,grid_size=(4,4)):
    prediction = model(test_input,training = False)
    # rescale ==
    predict = (prediction+1.0)/2.0
    prediction1 = predict.numpy()

    fig = plt.figure(figsize=(grid_size[1]*2,grid_size[0]*2))
    for i in range(predict.shape[0]):
        plt.subplot(grid_size[0],grid_size[1],i+1)
        img = predict[i,:,:,0]
        plt.imshow(img,cmap="gray")
        plt.axis("off")
    plt.suptitle(f"epochs {epochs}")
    file_name = os.path.join(output_dir,f"{epochs}.png")
    plt.savefig(file_name,bbox_inches="tight")
    plt.close(fig)




In [ ]:
Gen_and_save_iamge(generator,Epochs,inputs)


In [ ]:
# Buf_Size = 60000
# Bat_Size = 255
# Img_Hi = 28
# Img_Wi = 28
# Channels = 1
# Epochs = 30
# Lat_Dim = 100

In [ ]:
# Training Loop ==
@tf.function
def train(images):
    input = tf.random.normal([Bat_Size,Lat_Dim])

    with tf.GradientTape() as dis_tap,tf.GradientTape() as gen_tap:
        Generated_image = generator(input,training = False)

        real_image = discriminator(images,training = False)
        fake_image = discriminator(Generated_image,training = False)

        dis_loss = loss_discriminators(real_image,fake_image)
        gen_loss = loss_generators(fake_image)

    gra_of_dis = dis_tap.gradient(dis_loss,discriminator.trainable_variables)
    gra_of_gen = gen_tap.gradient(gen_loss,generator.trainable_variables)

    discriminator_optimizers.apply_gradients(zip(gra_of_dis,discriminator.trainable_variables))
    generator_optimizers.apply_gradients(zip(gra_of_gen,generator.trainable_variables))

    return gen_loss,dis_loss
def train1(datasets,epochs):
    st = time.time()
    for i in range(1,epochs+1):
        epochs_gen_loss = 0.0
        epochs_dis_loss = 0.0
        step = 0
        for j in datasets:
            g_loss,d_loss = train(j)
            epochs_gen_loss = g_loss.numpy()
            epochs_dis_loss = d_loss.numpy()
            step += 1
        epochs_gen_loss /= step
        epochs_dis_loss /= step
        if (i%5==0) or (i==1) or (i==epochs):
            Gen_and_save_iamge(generator,epochs,inputs)

        checkpoint_manager.save()
    Gen_and_save_iamge(generator,epochs,inputs)


In [ ]:
# Entry point
if __name__ == "__main__":
    if checkpoint_manager.latest_checkpoint:
        print("Restoring from checkpoint2:",checkpoint_manager.latest_checkpoint)
        checkpoint2.restore(checkpoint_manager.latest_checkpoint)
    train(train_dataset,Epochs)
    print(output_dir)
    print(checkpoint_dir)

TypeError: Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (<_PrefetchDataset element_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None)>, 30) and kwargs: {} for signature: (images).